In [7]:
import numpy as np 
import pandas as pd
from IPython.display import display
from annoy import AnnoyIndex
import warnings

warnings.filterwarnings('ignore')

pd.set_option('display.max_colwidth', None)

import os
for dirname, _, filenames in os.walk('Data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
import transformers
from tensorflow.keras.utils import plot_model

import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Concatenate, Dense, Flatten, Dot, Reshape, GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

import keras.backend as K
from keras.layers import Input, Dense, Embedding, Flatten, Lambda
from keras.models import Model
import numpy as np
import torch

import gc

# Explicitly collect garbage (may not free up all memory)
gc.collect()

90

In [8]:
gc.collect()

0

# Get Data

In [9]:
product_search_queries = [
    "Red dress for a wedding",
    "Best laptop under $1000",
    "Nike running shoes for women",
    "iPhone 13 reviews",
    "Men's winter jackets on sale",
    "Top-rated kitchen appliances",
    "Summer dresses for women",
    "Samsung TV specifications",
    "Gift ideas for anniversary",
    "XYZ brand headphones",
    "Women's shoes size 8",
    "Deals on smartphones",
    "Men's formal suits",
    "Fitness trackers with heart rate monitor",
    "Digital cameras with 4K video",
    "Wireless gaming mouse",
    "Outdoor camping gear",
    "Bluetooth earbuds under $50",
    "Designer handbags on clearance",
    "Gaming laptops with RTX graphics",
    "Organic skincare products",
    "Smartphones with long battery life",
    "Running shoes for flat feet",
    "Laptop deals for students",
    "Wireless noise-canceling headphones",
    "Digital cameras for beginners",
    "Best coffee makers under $50",
    "Latest fashion trends for summer",
    "Top-rated gaming keyboards",
    "Dining room furniture sets",
    "Winter coats for kids",
    "Men's watches with leather straps",
    "Home gym equipment for small spaces",
    "Women's handbags on sale",
    "Bluetooth speakers with waterproof features",
    "Outdoor patio furniture",
    "Affordable fitness trackers",
    "Cookware sets for induction cooktops",
    "Portable chargers for smartphones",
    "Desktop computers for gaming",
]

query_tower_cols = ['q0', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8',
                                  'q9', 'q10', 'q11', 'q12', 'q13', 'q14', 'q15', 'q16',
                                  'q17', 'q18', 'q19', 'q20', 'q21', 'q22', 'q23', 'q24',
                                  'q25', 'q26', 'q27', 'q28', 'q29', 'q30', 'q31']

product_tower_cols = ['p0', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10',
                      'p11', 'p12', 'p13', 'p14', 'p15', 'p16', 'p17', 'p18', 'p19', 'p20',
                      'p21', 'p22', 'p23', 'p24', 'p25', 'p26', 'p27', 'p28', 'p29', 'p30',
                      'p31', 'p32', 'p33', 'p34', 'p35', 'p36', 'p37', 'p38', 'p39', 'p40',
                      'p41', 'p42', 'p43', 'p44', 'p45', 'p46', 'p47', 'p48', 'p49', 'p50',
                      'p51', 'p52', 'p53', 'p54', 'p55', 'p56', 'p57', 'p58', 'p59', 'p60',
                      'p61', 'p62', 'p63', 'p64', 'p65', 'p66', 'p67', 'p68', 'p69', 'p70',
                      'p71', 'p72', 'p73', 'p74', 'p75', 'p76', 'p77', 'p78', 'p79', 'p80',
                      'p81', 'p82', 'p83', 'p84', 'p85', 'p86', 'p87', 'p88', 'p89', 'p90',
                      'p91', 'p92', 'p93', 'p94', 'p95', 'p96', 'p97', 'p98', 'p99', 'p100',
                      'p101', 'p102', 'p103', 'p104', 'p105', 'p106', 'p107', 'p108', 'p109', 'p110',
                      'p111', 'p112', 'p113', 'p114', 'p115', 'p116', 'p117', 'p118', 'p119', 'p120',
                      'p121', 'p122', 'p123', 'p124', 'p125', 'p126', 'p127', 'p128', 'p129', 'p130',
                      'p131', 'p132', 'p133', 'p134', 'p135', 'p136', 'p137', 'p138', 'p139', 'p140',
                      'p141', 'p142', 'p143', 'p144', 'p145', 'p146', 'p147', 'p148', 'p149', 'p150',
                      'p151', 'p152', 'p153', 'p154', 'p155', 'p156', 'p157', 'p158', 'p159'
                     ]

In [10]:
def prepare_data(query_tower_cols, product_tower_cols):
    df_product_embedding = pd.merge(
    pd.read_csv('/Users/ranjeetnagarkar/Desktop/Deep_learning_model/Data/product_150k.csv'),
    pd.read_parquet('/Users/ranjeetnagarkar/Desktop/Deep_learning_model/Data\
                    /shopping_queries_dataset_products.parquet')[['product_id',\
                                                                  'product_title']]\
                                                                    .drop_duplicates(),
    on = ['product_id']
    ).reset_index(drop=True)

    df_product_embedding['pid'] = range(0, df_product_embedding.shape[0])
    df_query_embedding = pd.read_csv('/Users/ranjeetnagarkar/Desktop/\
                                     Deep_learning_model/Data/query_150k.csv')
    df_query_embedding['qid'] = range(0, df_query_embedding.shape[0])

    df_dataset = pd.merge(pd.read_csv('/Users/ranjeetnagarkar/Desktop/Deep_learning_model\
                                      /Data/dataset_150k.csv'),
                      df_product_embedding[['product_id',\
                                            'product_title']].drop_duplicates(), \
                                                on = ['product_id']
                     )

    df_dataset['binary_label'] = df_dataset['esci_label'].\
        apply(lambda x: 1 if x == 'E' else 0)

    train_data = df_dataset[df_dataset['split'] != 'test']
    val_data = df_dataset[df_dataset['split'] == 'test']

    train_labels = np.array(train_data['binary_label'])
    val_labels = np.array(val_data['binary_label'])

    train_labels = train_labels.astype('float32')
    val_labels = val_labels.astype('float32')

    train_inputs = [
        np.array(train_data[query_tower_cols]),

        np.array(train_data[product_tower_cols])

    ]

    val_inputs = [
        np.array(val_data[query_tower_cols]),

        np.array(val_data[product_tower_cols])
    ]

    return train_inputs, train_labels, val_inputs, val_labels

train_inputs, train_labels, val_inputs, val_labels = \
    prepare_data(query_tower_cols, product_tower_cols)

## Create Query Model

In [12]:
import tensorflow as tf
from keras.layers import Input, Dense, Dot, Lambda
from keras.models import Model
from keras.callbacks import ModelCheckpoint

def build_model(query_tower_input_dim, product_tower_input_dim, embedding_dim=16):
    input_query = Input(shape=(query_tower_input_dim,), name='input_query')
    final_query_embedding = Dense(embedding_dim, \
                                  activation='linear', \
                                    name='embedding_layer_query')(input_query)
    normalized_query = Lambda(lambda x: tf.keras.backend.l2_normalize(x, axis=-1), \
                              name='normalize_query')(final_query_embedding)

    input_product = Input(shape=(product_tower_input_dim,), name='input_product')
    final_product_embedding = Dense(embedding_dim, activation='linear', \
                                    name='embedding_layer_product')(input_product)
    normalized_product = Lambda(lambda x: tf.keras.backend.l2_normalize(x, axis=-1), \
                                name='normalize_product')(final_product_embedding)

    cosine_similarity = Dot(axes=1, normalize=True, name='cosine_similarity')(\
        [normalized_product, normalized_query])

    model = Model(inputs=[input_query, input_product], \
                  outputs=cosine_similarity)

    checkpoint_path = 'best_model_weights.keras'  
    model_checkpoint = ModelCheckpoint(
        checkpoint_path,
        monitor='val_loss', 
        verbose=1,
        save_best_only=True,  
        mode='min' 
    )

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return input_query, input_product, model, model_checkpoint

embedding_dim=16
query_tower_input_dim = 32
product_tower_input_dim = 160
input_query, input_product, model, model_checkpoint\
      = build_model(query_tower_input_dim, product_tower_input_dim)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_product (InputLayer)  [(None, 160)]                0         []                            
                                                                                                  
 input_query (InputLayer)    [(None, 32)]                 0         []                            
                                                                                                  
 embedding_layer_product (D  (None, 16)                   2576      ['input_product[0][0]']       
 ense)                                                                                            
                                                                                                  
 embedding_layer_query (Den  (None, 16)                   528       ['input_query[0][0]']     

In [13]:
def fit_model(model, train_inputs, train_labels, val_inputs, val_labels, model_checkpoint, epochs=100, batch_size=64):
    # Fit the model with model checkpoint callback
    history = model.fit(
        # Training data and parameters
        train_inputs, train_labels, epochs=epochs, batch_size=batch_size,
        validation_data=(val_inputs, val_labels),
        callbacks=[model_checkpoint],
        # Giving positive class more importance to positive labels to ensure precision is high.
        class_weight=dict(enumerate([1,2]))
    )

    return history

history = fit_model(model, train_inputs, train_labels, val_inputs, val_labels, model_checkpoint)


Epoch 1/100
2124/2138 [============================>.] - ETA: 0s - loss: 0.8306 - accuracy: 0.6235
Epoch 1: val_loss improved from inf to 0.46182, saving model to best_model_weights.keras
2138/2138 [==============================] - 7s 3ms/step - loss: 0.8302 - accuracy: 0.6241 - val_loss: 0.4618 - val_accuracy: 0.8231
Epoch 2/100
2138/2138 [==============================] - ETA: 0s - loss: 0.7882 - accuracy: 0.6571
Epoch 2: val_loss did not improve from 0.46182
2138/2138 [==============================] - 5s 2ms/step - loss: 0.7882 - accuracy: 0.6571 - val_loss: 0.4855 - val_accuracy: 0.8468
Epoch 3/100
2123/2138 [============================>.] - ETA: 0s - loss: 0.7204 - accuracy: 0.7083
Epoch 3: val_loss did not improve from 0.46182
2138/2138 [==============================] - 7s 3ms/step - loss: 0.7204 - accuracy: 0.7085 - val_loss: 0.4889 - val_accuracy: 0.8094
Epoch 4/100
2134/2138 [============================>.] - ETA: 0s - loss: 0.6798 - accuracy: 0.7446
Epoch 4: val_loss did 

In [26]:
model.save('Recommendation model', save_format='tf')


In [27]:
model.save('recommendation_model.h5')

In [14]:
query_model = Model(inputs=[input_query, input_product],
                         outputs=model.get_layer('normalize_query').output
                   )

In [28]:
model = tf.keras.models.load_model('/content/Recommendation model')


# Sentence transformer

In [ ]:
# ! pip install nltk==3.8.1

In [ ]:
# ! pip install transformers==4.39.3


In [15]:
! pip install sentence-transformers==2.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 1.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [16]:
import tensorflow as tf
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained model (you can choose from various models like BERT, RoBERTa, etc.)
# model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
model_sentence_transformers = SentenceTransformer('distilbert-base-uncased')

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [17]:
def reshape_array(input_array, d):
    k, _ = input_array.shape
    new_array = np.zeros((k, d))

    for i in range(k):
        for j in range(d):
            start_idx = j * (768 // d)
            end_idx = (j + 1) * (768 // d) if j < (d - 1) else 768
            chunk = input_array[i, start_idx:end_idx]
            new_array[i, j] = np.mean(chunk)

    return new_array


In [18]:
def query_df_with_sentence_embeddings():

    query_dim = 32

    cols = ['q' + str(x) for x in list(range(0, query_dim))] + ['query']

    sentences = product_search_queries

    sentence_embeddings = model_sentence_transformers.encode(sentences, convert_to_tensor=True)

    sentence_embeddings = reshape_array(sentence_embeddings.cpu().numpy(), query_dim)

    df_query_test_actual = pd.DataFrame(sentence_embeddings)

    df_query_test_actual.columns = query_tower_cols
    df_query_test_actual['query'] = product_search_queries


    ###
    input_data_query = [

        np.array(df_query_test_actual[query_tower_cols]),
        np.array([np.array([0] * product_tower_input_dim)] * df_query_test_actual.shape[0]),

    ]
    query_embeddings = query_model.predict(input_data_query)

    cols = [f'q{x}'for x in range(embedding_dim)]

    df_query_test_output = pd.DataFrame(query_embeddings, columns = cols)
    df_query_test_output['query'] = product_search_queries

    return df_query_test_output


df_query_test = query_df_with_sentence_embeddings()

2/2 [==============================] - 0s 17ms/step


# Create product model

In [19]:
df_product_embedding = pd.merge(
    pd.read_csv('drive/MyDrive/Data/product_150k.csv'),
    pd.read_parquet('drive/MyDrive/Data/shopping_queries_dataset_products.parquet')[['product_id','product_title']].drop_duplicates(),
    on = ['product_id']
).reset_index(drop=True)

df_product_embedding['pid'] = range(0, df_product_embedding.shape[0])

In [20]:
product_model = Model(inputs=[input_query, input_product],
                         outputs=model.get_layer('normalize_product').output
                   )


input_data_product = [
    np.array([np.array([0] * query_tower_input_dim)] * df_product_embedding.shape[0]),

    np.array(df_product_embedding[product_tower_cols])
]

product_embeddings = product_model.predict(input_data_product)
len(product_embeddings)

4615/4615 [==============================] - 6s 1ms/step


147673

In [21]:
df_product_embeddings_model = pd.DataFrame(product_embeddings, columns = [f'p{x}'for x in range(embedding_dim)] )

df_product_embeddings_model['product_id'] = df_product_embedding['product_id']
df_product_embeddings_model['product_title'] = df_product_embedding['product_title']
df_product_embeddings_model['pid'] = df_product_embedding['pid']

df_product_embeddings_model.head()

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,product_id,product_title,pid
0,0.081530,0.429039,0.290590,-0.565156,0.107100,-0.083553,-0.114021,0.347309,-0.052731,0.173256,-0.052642,0.087726,0.291771,0.313199,0.151087,0.063575,B01A5DU2TA,"Crocs Kids' Handle It Rain Boots , Candy Pink, 8 Toddler",0
1,0.099167,0.493109,0.340570,-0.421917,0.176736,-0.297714,-0.007500,0.326943,-0.024755,0.260635,0.065543,0.174644,0.233466,0.129666,0.226021,-0.021102,B07346RPKN,"Hatley Kids' Little Classic Rain Boots, Pink & Navy, 8 US Child",1
2,0.259446,0.448643,0.247448,-0.458677,0.089554,-0.163494,-0.123989,0.510153,0.198851,0.184204,-0.203165,0.023060,0.140381,0.100708,-0.009139,0.064797,B078TCVZ4V,Outee Rubber Kids Rain Boots,2
3,-0.116439,0.284625,0.081032,-0.492689,-0.001776,-0.209153,-0.258369,0.035894,-0.236697,0.387407,0.175222,0.241779,0.148811,0.409855,-0.242743,0.003473,B07BR3PPDR,Hope & Henry Girls' Red Milano Stitch Cardigan,3
4,0.198218,0.447664,0.176400,-0.311154,0.130842,-0.268595,-0.130478,0.209426,-0.323508,0.222097,0.084741,0.142367,0.435183,0.325229,-0.032700,0.067561,B07H5C4P19,Spring&Gege Youth Solid Full Zipper Hoodies Soft Kids Hooded Sweatshirt for Boys and Girls Size 3-4 Years Red,4


# Annoy Index

In [30]:
# pm = AnnoyIndex(embedding_dim, 'euclidean')
pm = AnnoyIndex(embedding_dim, 'dot')
mp_product_dict = {}

for ix,row in df_product_embeddings_model.iterrows():
    mp_product_dict[int(row['pid'])] = row['product_title']

    key = int(row['pid'])
    vec = list(row[['p'+str(x) for x in list(range(embedding_dim))]])

#     print(key,vec)
    pm.add_item(key,vec)

In [31]:
pm.build(100) # 100 trees
pm.save('product_model.tree')

True

In [ ]:
# pm = AnnoyIndex(embedding_dim,  'euclidean')
pm = AnnoyIndex(embedding_dim,  'dot')
pm.load('product_model.tree')

# Print Result

In [32]:
df_query_test

,q0,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,q11,q12,q13,q14,q15,query
0,0.123872,0.018339,-0.191966,-0.406441,0.456101,-0.251823,-0.216898,-0.080171,-0.085951,0.235478,0.083408,-0.041833,0.252370,0.429993,-0.078652,-0.361998,Red dress for a wedding
1,-0.030445,-0.255088,-0.178845,-0.395668,0.112950,0.023204,0.198216,-0.370191,-0.371969,0.129430,0.403274,-0.368063,0.021416,0.150258,0.239006,-0.149879,Best laptop under $1000
2,0.112434,0.305571,-0.187771,-0.431009,-0.057991,-0.085707,-0.080083,-0.098515,-0.154227,0.371568,0.559796,0.068757,0.250735,0.180003,-0.126144,-0.234524,Nike running shoes for women
3,0.259584,-0.174980,-0.157235,-0.277748,0.137736,-0.080742,-0.067975,-0.248135,-0.361062,0.183975,0.337590,-0.175796,0.481146,0.261208,0.087269,-0.303345,iPhone 13 reviews
4,0.004551,0.592728,-0.135330,-0.375212,0.183430,-0.190335,-0.166496,0.074352,-0.205505,0.125151,0.207071,0.076028,0.271425,0.084676,-0.429667,-0.120091,Men's winter jackets on sale
5,0.017177,0.007392,0.124211,-0.463751,0.191960,0.197132,0.132752,0.077927,-0.077138,0.135357,0.329161,-0.273813,-0.137077,0.155039,0.315029,-0.565764,Top-rated kitchen appliances
6,0.026442,0.271571,0.004528,-0.315838,0.291964,-0.389523,-0.220104,-0.080848,-0.252442,0.177467,0.401123,-0.008218,0.427437,0.208270,-0.144469,-0.175168,Summer dresses for women
7,0.133294,-0.187916,-0.323494,-0.272277,0.217369,-0.035626,-0.064391,-0.411004,-0.463751,0.150222,0.221327,-0.236615,0.387749,0.202400,0.090820,-0.066188,Samsung TV specifications
8,0.311206,-0.146117,0.038525,-0.365762,0.308038,-0.178839,-0.146073,0.075145,-0.204811,-0.003451,0.069755,-0.093449,0.289381,0.014346,0.673154,-0.006230,Gift ideas for anniversary
9,0.217082,-0.238909,-0.116851,-0.227248,0.147729,0.371619,0.030548,-0.169227,-0.325932,0.480970,0.054084,-0.161465,0.232138,0.336867,0.327221,0.002112,XYZ brand headphones


In [24]:
results = []
top_k = 20
embedding_dim = 16

desc = []
for i in range(top_k):
    desc.append(f'product_title_{(i+1)}')
desc = ['query'] + desc

for ix,row in df_query_test.iterrows():
    qvec = [f'q{x}' for x in list(range(embedding_dim))]
    qvec = row[qvec]
    similar_vector_ids,similar_distances = pm.get_nns_by_vector(qvec, top_k, include_distances=True)

#     print(similar_vector_ids, similar_distances)

    similar_vector_product_title = [mp_product_dict[x] for x in similar_vector_ids]
    results.append([row['query']] + similar_vector_product_title)

df_results = pd.DataFrame(results, columns =  desc)

df_results.head(5)

,query,product_title_1,product_title_2,product_title_3,product_title_4,product_title_5,product_title_6,product_title_7,product_title_8,product_title_9,...,product_title_11,product_title_12,product_title_13,product_title_14,product_title_15,product_title_16,product_title_17,product_title_18,product_title_19,product_title_20
0,Red dress for a wedding,Yanstar Handmade Crystal Bridal Belts Sashes Navy Blue Wedding Belt With Rhinestones For Wedding Bridesmaid Dress (Silver-Navy),"Bride Shirt - Rhinestone Diamond Bride T-Shirt - Wedding, Engagement & Bachelorette Party Tee Shirt - X-Large - White Tee(DiamBrd RS) Wht/XL","6-Pack Cheers Btches and Future Mrs Bachelorette Shot Glass Necklace - Hot Pink and White with Gold Foil Bridal Party Necklaces, Bachelorette Party Supplies, 30 Inches Long",Yanstar White Sash Crystal Applique Wedding Bridal Belts In Silver With Pearls Beaded On Wedding Prom Dress-7.7In2In,FUMUD Silver Color Baroque Vintage Eiffel Tower Diadem Wedding Crystal Rhinestone Tiaras Bridal Pearl Crowns Headband For Bride Women (Silver),"Muses Art Design Birthstone Eternity Band Ring (Channel Set/Full Round), Stackable Fashion Ring with Swarovski Crystal Birthstone",Mardi Gras Theme Photography Backdrop Masquerade Backgrounds Birthday Dancing Party Photo Booth for Wedding Bachelorette Party Decorations Banner (7X5FT) D061,Flower Girl Dress Curly V-Neck Rose Embroidery Allover for Big Girl Red 12 JKS.2066,"Aofur Womens Evening Dress Ball Gown Prom Party Wedding Formal Long Chiffon Maxi Dresses Plus Size (XXXX-Large, White)",...,Afco Womens Crown Eiffel Tower Rhinestone Royal Bridal Tiara Jewelry For Wedding Party (Golden),"Eurotard Womens 13524 Long Sleeve Worship Praise Liturgical Full Dance Dress (White, Medium)",Weddingtopia Crystal Rose Gold Wedding Tiara And Matching Rhinestone Wedding Necklace Set Rhinestone Bridal Tiara Crown with side combs Wedding Hairband Comb Pin (ROSE GOLD PEACH),yanstar Wedding Bridal Garter Belt Champange Stretch Lace Bridal Garter Sets with Rose Gold Rhinestones Clear Crystal Pearl for Wedding,Fdesigner Bride Cathedral Veil Wedding Chapel Veils Bridal Headpieces Statement Veils Long Soft Veil with Comb 2T (White),JAEDEN Prom Dresses Long Formal Evening Gowns Off Shoulder Prom Dress Tulle Lace Appliques Evening Party Dress Champagne,InsNova Women’s Spring Burgundy Cocktail Dress for Summer Teen Girls Graduation Wedding Guest Dinner Party Beach,"Women's Tuxedo Gothic Tailcoat Jacket Steampunk VTG Victorian Coat Wedding Uniform (XL, Black)","Aofur Womens V Neck Chiffon Casual Maxi Dress Wedding Evening Gowns Summer Sleeveless A-Line Party Long Skirt (X-Large, Burgundy)",BriLove Gold-Toned Dangle Earrings for Women Wedding Bridal Bohemian Boho Crystal Multiple Teardrop Chandelier Long Earrings Royal Blue Sapphire Color
1,Best laptop under $1000,Factory Original SONY RMT-TX300P Universal Replacement TV Remote Control Substitute RMF-TX200U RMF-TX300U RMF-TX201U RMF-TX310U RMF-TX220U RMF-TX300B RMF-TX310B RMF-TX200B RMT-TX102U RMT-TX100U,"70mai Dash Cam Pro Plus+ A500S, 2.7K Ultra Full HD, Built in WiFi GPS Smart Dash Camera for Cars, ADAS, Sony IMX335, 2'' IPS LCD Screen, 140° FOV, WDR, Night Vision","[Upgraded] ESR Wireless Charger, Metal Frame Ultra-Thin Fast Charging Pad, 7.5W for iPhone 12/Mini/Pro/Pro Max/11/SE, 10W Fast Charging for Galaxy S20/S20 Ultra/Note10, Pixel 3/4, Black","Mini DisplayPort to DVI Cable, Benfei Mini DisplayPort to DVI 6 Feet Cable (Thunderbolt 2 Compatible) with MacBook Air/Pro, Surface Pro/Dock, Monitor, Projector",Benfei Mini Displayport → DVI 変換 ケーブル 1.8m ブラック【相性保証付き】| デュアル ディスプレイ 対応 堅牢化仕様 金メッキ コネクタ シングルリンク パッシブタイプ,"Mini DisplayPort to HDMI, Benfei Mini DP to HDMI 4K Adapter (Thunderbolt Compatible) Gold-Plated Cord Compatible for MacBook Pro, MacBook Air, Mac Mini, Microsoft Surface Pro 3/4","Satechi Flexible Apple MFi Certified Lightning USB Charging Cable - Compatible with iPhone 12 Pro Max/12 Pro/12 Mini/12, 11 Pro Max/11 Pro/11, Xs Max/XS/XR/X, 8 Plus/8 (6-Inch,